## HBCU  
### by Erika Harrell
### This data visualization project eplores data on Historically Black Colleges and Universities in the U.S. (HBCUs) from the U.S. Department of Education downloaded from National Center for Education Statistics website on March 28, 2024.

In [31]:
#show all output in each cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
#import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import regex as re
from geopy.geocoders import Nominatim
#read in dataset
df = pd.read_csv('hbcu.csv')
#print out column names
print(df.columns)

Index(['Name', 'Address', 'Website', 'Type', 'Awards offered',
       'Campus setting', 'Campus housing', 'Student population',
       'Undergraduate students', 'Graduation Rate', 'Transfer-Out Rate',
       'Cohort Year *', 'Net Price **', 'Largest Program', 'IPEDS ID',
       'OPE ID'],
      dtype='object')


In [32]:
#get data type for each column
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Name                    102 non-null    object 
 1   Address                 105 non-null    object 
 2   Website                 102 non-null    object 
 3   Type                    102 non-null    object 
 4   Awards offered          102 non-null    object 
 5   Campus setting          101 non-null    object 
 6   Campus housing          102 non-null    object 
 7   Student population      100 non-null    float64
 8   Undergraduate students  97 non-null     float64
 9   Graduation Rate         96 non-null     object 
 10  Transfer-Out Rate       63 non-null     object 
 11  Cohort Year *           102 non-null    object 
 12  Net Price **            96 non-null     object 
 13  Largest Program         102 non-null    object 
 14  IPEDS ID                102 non-null    fl

In [33]:
#check Name column for missing data
df['Name'].isna().sum()
#remove rows with missing Names
df = df[df['Name'].notna()]
#check again for missing Names
df['Name'].isna().sum()

3

0

In [34]:
#create new highest degree column
#get unique values for 'Awards Offered' column
df['Awards offered'].unique()

array(["Bachelor's degree|Master's degree|Post-master's certificate|Doctor's degree - research/scholarship",
       "Bachelor's degree|Postbaccalaureate certificate|Master's degree|Post-master's certificate|Doctor's degree - research/scholarship|Doctor's degree - professional practice",
       "Less than one year certificate|One but less than two years certificate|Associate's degree|Bachelor's degree|Master's degree|Post-master's certificate",
       "Associate's degree|Bachelor's degree|Master's degree|Post-master's certificate|Doctor's degree - professional practice",
       "Associate's degree|Bachelor's degree|Master's degree|Doctor's degree - research/scholarship",
       "Associate's degree|Bachelor's degree",
       "Bachelor's degree|Master's degree", "Bachelor's degree",
       "Bachelor's degree|Postbaccalaureate certificate|Master's degree",
       "Less than one year certificate|One but less than two years certificate|Associate's degree",
       "Less than one year certific

In [35]:
#find NaN values in Awards offered column
df['Awards offered'].isna().sum()

0

In [36]:
#creating binary variables for each degree type
df['Doctor'] = df['Awards offered'].str.contains('Doctor')
df['Master'] = df['Awards offered'].str.contains('Master')
df['Bachelor'] = df['Awards offered'].str.contains('Bachelor')
df['Associate'] = df['Awards offered'].str.contains('Associate')

In [37]:
#create Highest Degree column
df.loc[df['Associate'] == True, 'Highest Degree'] = 'Associate\'s degree' 
df.loc[df['Bachelor'] == True, 'Highest Degree'] = 'Bachelor\'s degree' 
df.loc[df['Master'] == True, 'Highest Degree'] = 'Master\'s degree' 
df.loc[df['Doctor'] == True, 'Highest Degree'] = 'Doctoral degree' 
df['Highest Degree'].value_counts()

Highest Degree
Doctoral degree       40
Master's degree       33
Bachelor's degree     18
Associate's degree    11
Name: count, dtype: int64

In [38]:
#remove binary columns used to create Highest degree column
df = df.drop(['Associate','Bachelor','Master', 'Doctor'], axis=1)
#Make sure columns are dropped
df.columns

Index(['Name', 'Address', 'Website', 'Type', 'Awards offered',
       'Campus setting', 'Campus housing', 'Student population',
       'Undergraduate students', 'Graduation Rate', 'Transfer-Out Rate',
       'Cohort Year *', 'Net Price **', 'Largest Program', 'IPEDS ID',
       'OPE ID', 'Highest Degree'],
      dtype='object')

In [39]:
#pie chart for percentage of schools by highest degree offered
fig = px.pie(df, names=df['Highest Degree'],
                   title='Percentage of HBCUs by the Highest Degree Offered',color_discrete_sequence=px.colors.sequential.RdBu)
fig.show()

In [40]:
#create Private/Public column
#get unique values for Type column
df['Type'].unique()
#create mapping from Type variable to Public/Private column
type_mapping = {'4-year, Public':'Public HBCU',
                '2-year, Public':'Public HBCU', 
                '4-year, Private not-for-profit':'Private HBCU',
                '2-year, Private not-for-profit':'Private HBCU',
                "4-year, primarily associate's, Private not-for-profit": 'Private HBCU'}
#recode Type column into Public/Private column
df = df.assign(PublicPrivate=df.Type.map(type_mapping))
#crosstab to see if recode went correctly
pd.crosstab(index= df['Type'], columns=df['PublicPrivate'], margins=True)

array(['4-year, Public', '4-year, Private not-for-profit',
       '2-year, Public', '2-year, Private not-for-profit',
       "4-year, primarily associate's, Private not-for-profit"],
      dtype=object)

PublicPrivate,Private HBCU,Public HBCU,All
Type,,,
"2-year, Private not-for-profit",1,0,1
"2-year, Public",0,10,10
"4-year, Private not-for-profit",48,0,48
"4-year, Public",0,42,42
"4-year, primarily associate's, Private not-for-profit",1,0,1
All,50,52,102


In [41]:
#create histogram for Highest degree numbers for public and private HBCUs
fig = px.histogram(df, df['Highest Degree'],
                   title='Number of HBCUs by the Highest Degree Offered and Type of Institution',
                   color=df['PublicPrivate'],
                   color_discrete_sequence=px.colors.qualitative.Pastel
                   )
fig.show()

In [42]:
#student population
#look for schools with missing data on student population
df['Name'][df['Student population'].isna()]
#fill in missing information
df.loc[df['Name']=='Southern University Law Center', 'Student population']=843
df.loc[df['Name']=='University of the Virgin Islands-Albert A. Sheen', 'Student population']=2138
#check that no schools have missing data on student population
df['Name'][df['Student population'].isna()]

77                      Southern University Law Center
92    University of the Virgin Islands-Albert A. Sheen
Name: Name, dtype: object

Series([], Name: Name, dtype: object)

In [43]:
#create histogram for student population
fig =px.histogram(df, x=df['Student population'], title='HBCU student population', color_discrete_sequence=['indianred']) 
fig.show()

In [44]:
#create scatterplot of student population by public/private HBCUs
fig = px.scatter(df,x=df['Student population'],
                 size=df['Student population'],
                 hover_name=df['Name'],
                 title='Student population of Public and Private HBCUs', 
                 color_discrete_sequence=px.colors.qualitative.Bold,
                 color=df["PublicPrivate"])
fig.show()

In [45]:
#create box plots for student population by highest degree offered
fig = px.box(df, x=df['Highest Degree'], y=df['Student population'], title = 'HBCU Student population by the Highest Degree Offered',color_discrete_sequence=['goldenrod'])
fig.show()

In [46]:
#percentage of student population that were undergraduates column
df['Percent Undergraduates'] = (df['Undergraduate students']/df['Student population'])*100

In [47]:
#create scatterplot of percentage of student population that are undergraduates for public and private HBCUs
fig = px.scatter(df,x=df['Percent Undergraduates'],
                hover_name=df['Name'],
                 title='Percentage of the Student Population that are Undergraduates for Public and Private HBCUs', 
                 color_discrete_sequence=px.colors.qualitative.D3, labels={'Percent Undergraduates':"Percent of students that are undergraduates"},
                 color=df["PublicPrivate"])
fig.show()

In [48]:
#collapsing Campus setting column
#rename Campus setting column to get rid of space in columns name
df = df.rename(columns={'Campus setting': 'Campus_setting'})
#get unique values for Campus setting column
df['Campus_setting'].unique()
#find which schools are missing on Campus setting
df['Name'][df['Campus_setting'].isna()]
#replace na value with Town
df['Campus_setting'].fillna('Town: Distant')
#check unique values for Campus setting column
df['Campus_setting'].unique()

array(['City: Midsize', 'City: Small', 'Rural: Remote', 'City: Large',
       'Town: Distant', 'Suburb: Large', 'Rural: Fringe', 'Town: Fringe',
       'Rural: Distant', 'Suburb: Midsize', 'Town: Remote', nan],
      dtype=object)

92    University of the Virgin Islands-Albert A. Sheen
Name: Name, dtype: object

0        City: Midsize
1        City: Midsize
2          City: Small
3        Rural: Remote
4        City: Midsize
            ...       
97     Suburb: Midsize
98       Rural: Fringe
99       Town: Distant
100      City: Midsize
101        City: Large
Name: Campus_setting, Length: 102, dtype: object

array(['City: Midsize', 'City: Small', 'Rural: Remote', 'City: Large',
       'Town: Distant', 'Suburb: Large', 'Rural: Fringe', 'Town: Fringe',
       'Rural: Distant', 'Suburb: Midsize', 'Town: Remote', nan],
      dtype=object)

In [49]:
#create mapping from Campus setting column to Setting column
type_mapping = {'City: Midsize':'City',
                'City: Small': 'City', 
                'Rural: Remote':'Rural',
                'City: Large':'City',
                'Town: Distant':'Town',
                'Suburb: Large': 'Suburb',
                'Rural: Fringe':'Rural',
                'Town: Fringe': 'Town',
                'Rural: Distant': 'Rural',
                'Suburb: Midsize' : 'Suburb',
                'Town: Remote': 'Town'
               }
#recode Campus setting column into Setting column
df = df.assign(Setting=df.Campus_setting.map(type_mapping))
#crosstab to see if recode ran correctly
pd.crosstab(index= df['Campus_setting'], columns=df['Setting'], margins=True)

Setting,City,Rural,Suburb,Town,All
Campus_setting,,,,,
City: Large,30,0,0,0,30
City: Midsize,25,0,0,0,25
City: Small,13,0,0,0,13
Rural: Distant,0,2,0,0,2
Rural: Fringe,0,4,0,0,4
Rural: Remote,0,1,0,0,1
Suburb: Large,0,0,6,0,6
Suburb: Midsize,0,0,2,0,2
Town: Distant,0,0,0,14,14


In [50]:
#create boxplots of student population for each setting
fig = px.box(df, x=df['Setting'], y=df['Student population'], color_discrete_sequence=['navy'], title='HBCU student populations by Setting')
fig.show()

In [51]:
#create scatterplot of student populations by setting
fig = px.scatter(df,x=df['Student population'],
                 size=df['Student population'],
                 hover_name=df['Name'],
                 title='Student population of HBCUs by Setting', 
                 color_discrete_sequence=px.colors.qualitative.Dark24,
                 color=df["Setting"])
fig.show()

In [52]:
#cleaning up Address column to prep for geocoding
#make list out of Address column
Address1 = df['Address'].to_list()
#replace extra 4 digits of zip codes with blanks in addresses using regular expressions
pattern1 = '-[0-9]+'
repl1 = ''
Address2 = []
for index, x in enumerate(Address1):
    result = re.sub(pattern1, repl1, x)
    Address2.append(result)
#clean address for Arkansas Baptist College
print(Address2[6])
Address2[6] = '1621 Dr. Martin Luther King Drive, Little Rock, Arkansas 72202'
print(Address2[6])

1600 Dr. Martin Luther King Jr. Drive, Little Rock, Arkansas 72202
1621 Dr. Martin Luther King Drive, Little Rock, Arkansas 72202


In [53]:
#clean Name1 list to prep for geocoding
#make list out of Name column
Name1 = df['Name'].to_list()
#clean up name for Jarvis Christian
print(Name1[40])
Name1[40] = 'Jarvis Christian College'
print(Name1[40])
#clean up name for North Carolina A&T State University
print(Name1[58])
Name1[58] = 'North Carolina A&T State University'
print(Name1[58])

Jarvis Christian University
Jarvis Christian College
North Carolina A & T State University
North Carolina A&T State University


In [54]:
#geocode addresses and names of schools to get coordinates
geo=Nominatim(user_agent="hbcu")
#create empty lists to store latitudes and longitudes
Latitude=[]
Longitude=[]
#geocode using Name1 and Address1 lists to put coordinates into latitude and longitude lists
for index,x in enumerate(Name1):
    l1 = geo.geocode(x, timeout=None)
    if l1 is None:
        l1=geo.geocode(Address2[index], timeout=None)
    Latitude.append(l1.latitude)
    Longitude.append(l1.longitude)

In [55]:
#look for None values in in Latitude and Longitude lists
res_lat = [i for i, val in enumerate(Latitude) if val == None]
print(res_lat)
res_lon = [i for i, val in enumerate(Longitude) if val == None]
print(res_lon)

[]
[]


In [56]:
#add longitude and latitude lists to the data frame as columns
df['Longitude'] = Longitude
df['Latitude'] = Latitude
#check columns
print(df.columns)

Index(['Name', 'Address', 'Website', 'Type', 'Awards offered',
       'Campus_setting', 'Campus housing', 'Student population',
       'Undergraduate students', 'Graduation Rate', 'Transfer-Out Rate',
       'Cohort Year *', 'Net Price **', 'Largest Program', 'IPEDS ID',
       'OPE ID', 'Highest Degree', 'PublicPrivate', 'Percent Undergraduates',
       'Setting', 'Longitude', 'Latitude'],
      dtype='object')


In [57]:
#bubble map for student population
#map uses Latitude and Longitude columns to locate schools
#hover text includes school name
#color and size of dots indicate student population
fig = px.scatter_mapbox(df, lat=df['Latitude'], lon=df['Longitude'], text=df['Name'], size=df['Student population'],color=df['Student population'],
                        zoom=4.5, mapbox_style='open-street-map', color_continuous_scale='hot', title='HBCU student population map', height=800)
fig.show()

In [58]:
#bubble map for public and private HBCUs
#map uses Latitutde and Longitude columns to locate schools
#color of dots indicate public or private HBCU
#hover text has school name
fig = px.scatter_mapbox(df, lat=df['Latitude'], lon=df['Longitude'], text=df['Name'], color=df['PublicPrivate'], color_discrete_sequence=px.colors.qualitative.Dark24_r,
                        zoom=4.5, mapbox_style='open-street-map', title='Map of HBCUs by Type', height=800)
fig.show()

In [59]:
#creating a state column using Address column
df.loc[df['Address'].str.contains('Alabama'), 'State'] = 'Alabama' 
df.loc[df['Address'].str.contains('Georgia'), 'State'] = 'Georgia' 
df.loc[df['Address'].str.contains('Florida'), 'State'] = 'Florida'
df.loc[df['Address'].str.contains('Tennessee'), 'State'] = 'Tennessee'  
df.loc[df['Address'].str.contains('Mississippi'), 'State'] = 'Mississippi'  
df.loc[df['Address'].str.contains('Texas'), 'State'] = 'Texas'
df.loc[df['Address'].str.contains('Arkansas'), 'State'] = 'Arkansas'    
df.loc[df['Address'].str.contains('Oklahoma'), 'State'] = 'Oklahoma'
df.loc[df['Address'].str.contains('Missouri'), 'State'] = 'Missouri'
df.loc[df['Address'].str.contains('Ohio'), 'State'] = 'Ohio'
df.loc[df['Address'].str.contains('Virginia'), 'State'] = 'Virginia'
df.loc[df['Address'].str.contains('South Carolina'), 'State'] = 'South Carolina'
df.loc[df['Address'].str.contains('North Carolina'), 'State'] = 'North Carolina'
df.loc[df['Address'].str.contains('Maryland'), 'State'] = 'Maryland'
df.loc[df['Address'].str.contains('Delaware'), 'State'] = 'Delaware'
df.loc[df['Address'].str.contains('Pennsylvania'), 'State'] = 'Pennsylvania'
df.loc[df['Address'].str.contains('Kentucky'), 'State'] = 'Kentucky'
df.loc[df['Address'].str.contains('Louisiana'), 'State'] = 'Louisiana'     
df.loc[df['Address'].str.contains('District of Columbia'), 'State'] = 'District of Columbia'  
df.loc[df['Address'].str.contains('Virgin Islands'), 'State'] = 'Virgin Islands'    
#check for missing cases           
df['State'].isna().sum()
df['Address'][df['State'].isna()]

0

Series([], Name: Address, dtype: object)

In [60]:
#histogram of HBCUs by state
#uses State column
fig = px.histogram(df, x="State", title='Number of HBCUs by state',
                   category_orders=dict(State=["Alabama", "Arkansas", "Delaware", "District of Columbia", "Florida",
                                               "Georgia", "Kentucky","Louisiana","Maryland", "Mississippi","Missouri","North Carolina","Ohio",
                                               "Oklahoma","Pennsylvania",
                                               "South Carolina", "Tennessee", "Texas", "Virgin Islands", "Virginia"]),
                                               color_discrete_sequence=['tan'])
fig.show()
